In [1]:
import numpy as np
from scipy.linalg import svd

A = np.array([[1, 2], [3, 4], [5, 6]])
print(A)

[[1 2]
 [3 4]
 [5 6]]


In [3]:
U, S, VT = svd(A)
print(U)
print("-=---------------------------------------=-")
print(S)
print("-=---------------------------------------=-")
print(VT)

[[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]
-=---------------------------------------=-
[9.52551809 0.51430058]
-=---------------------------------------=-
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]


In [4]:
import re
import numpy as np
import pandas as pd
from scipy import linalg, spatial
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, SparsePCA, TruncatedSVD
from sklearn.feature_extraction.text import (CountVectorizer, TfidfTransformer, TfidfVectorizer)
from sklearn.cluster import KMeans

from sklearn.utils.extmath import randomized_svd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
corpus = [

          "With all of the critical success Downey had experienced throughout his career, he had not appeared in a blockbuster film. That changed in 2008 when Downey starred in two critically and commercially successful films, Iron Man and Tropic Thunder. In the article Ben Stiller wrote for Downey's entry in the 2008 edition of The Time 100, he offered an observation on Downey's commercially successful summer at the box office.",
          "On June 14, 2010, Downey and his wife Susan opened their own production company called Team Downey. Their first project was The Judge.",
          "Robert John Downey Jr. is an American actor, producer, and singer. His career has been characterized by critical and popular success in his youth, followed by a period of substance abuse and legal troubles, before a resurgence of commercial success in middle age.",
          "In 2008, Downey was named by Time magazine among the 100 most influential people in the world, and from 2013 to 2015, he was listed by Forbes as Hollywood's highest-paid actor. His films have grossed over $14.4 billion worldwide, making him the second highest-grossing box-office star of all time."
          
          ]

stop_words = set(stopwords.words('english'))

["With all of the critical success Downey had experienced throughout his career, he had not appeared in a blockbuster film. That changed in 2008 when Downey starred in two critically and commercially successful films, Iron Man and Tropic Thunder. In the article Ben Stiller wrote for Downey's entry in the 2008 edition of The Time 100, he offered an observation on Downey's commercially successful summer at the box office.",
 'On June 14, 2010, Downey and his wife Susan opened their own production company called Team Downey. Their first project was The Judge.',
 'Robert John Downey Jr. is an American actor, producer, and singer. His career has been characterized by critical and popular success in his youth, followed by a period of substance abuse and legal troubles, before a resurgence of commercial success in middle age.',
 "In 2008, Downey was named by Time magazine among the 100 most influential people in the world, and from 2013 to 2015, he was listed by Forbes as Hollywood's highest-

In [16]:
filtered_document= []
filtered_text = []

for document in corpus:
    
    clean_document = " ".join(re.sub(r"[^A-Za-z \—]+", " ", document).split())
    
    document_tokens = word_tokenize(clean_document)
    
    for word in document_tokens:
        if word not in stop_words:
            filtered_document.append(word)

    filtered_text.append(' '.join(filtered_document))

In [17]:
vectorizer = CountVectorizer()

counts_matrix = vectorizer.fit_transform(filtered_text)

feature_names = vectorizer.get_feature_names()

count_matrix_df = pd.DataFrame(counts_matrix.toarray(), columns=feature_names)
count_matrix_df.index = ['Document 1','Document 2','Document 3','Document 4']

print("Word frequency matrix: \n", count_matrix_df)

Word frequency matrix: 
             abuse  actor  age  american  among  appeared  article  ben  \
Document 1      0      0    0         0      0         1        1    1   
Document 2      0      0    0         0      0         1        1    1   
Document 3      1      1    1         1      0         1        1    1   
Document 4      1      2    1         1      1         1        1    1   

            billion  blockbuster  ...  time  tropic  troubles  two  wife  \
Document 1        0            1  ...     1       1         0    1     0   
Document 2        0            1  ...     1       1         0    1     1   
Document 3        0            1  ...     1       1         1    1     1   
Document 4        1            1  ...     3       1         1    1     1   

            with  world  worldwide  wrote  youth  
Document 1     1      0          0      1      0  
Document 2     1      0          0      1      0  
Document 3     1      0          0      1      1  
Document 4     1   

In [18]:
vectorizer = TfidfVectorizer(stop_words=stop_words,max_features=10000, max_df = 0.5,
                                    use_idf = True,
                                    ngram_range=(1,3))

X = vectorizer.fit_transform(filtered_text)
print(X.shape)
print(feature_names)

num_clusters = 4

km = KMeans(n_clusters=num_clusters)
km.fit(X)
    
clusters = km.labels_.tolist()
print(clusters)

(4, 139)
['abuse', 'actor', 'age', 'american', 'among', 'appeared', 'article', 'ben', 'billion', 'blockbuster', 'box', 'called', 'career', 'changed', 'characterized', 'commercial', 'commercially', 'company', 'critical', 'critically', 'downey', 'edition', 'entry', 'experienced', 'film', 'films', 'first', 'followed', 'forbes', 'grossed', 'grossing', 'highest', 'his', 'hollywood', 'in', 'influential', 'iron', 'john', 'jr', 'judge', 'june', 'legal', 'listed', 'magazine', 'making', 'man', 'middle', 'named', 'observation', 'offered', 'office', 'on', 'opened', 'paid', 'people', 'period', 'popular', 'producer', 'production', 'project', 'resurgence', 'robert', 'second', 'singer', 'star', 'starred', 'stiller', 'substance', 'success', 'successful', 'summer', 'susan', 'team', 'that', 'the', 'their', 'throughout', 'thunder', 'time', 'tropic', 'troubles', 'two', 'wife', 'with', 'world', 'worldwide', 'wrote', 'youth']
[0, 0, 2, 1]


<ipython-input-18-c97f33d4e72f>:12: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  km.fit(X)


In [20]:
U, Sigma, VT = randomized_svd(X, n_components=10, n_iter=100, random_state=122)

svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)
    
print(U.shape)

for i, comp in enumerate(VT):
    terms_comp = zip(feature_names, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Cluster "+str(i)+": ")
    for t in sorted_terms:
        print(t)
    print(" ")

(4, 4)
Cluster 0: 
('american', 0.14737056548337124)
('abuse', 0.10691121146062435)
('actor', 0.10691121146062427)
('article', 0.10691121146062425)
('ben', 0.10691121146062425)
('billion', 0.10691121146062425)
('called', 0.10691121146062425)
 
Cluster 1: 
('offered', 0.21107535300139604)
('among', 0.10553767650069802)
('appeared', 0.10553767650069802)
('blockbuster', 0.10553767650069802)
('box', 0.10553767650069802)
('characterized', 0.10553767650069802)
('commercial', 0.10553767650069802)
 
Cluster 2: 
('abuse', 0.1494515724826932)
('offered', 0.021436190697647406)
('article', 0.012740642002274236)
('ben', 0.012740642002274236)
('billion', 0.012740642002274236)
('called', 0.012740642002274236)
('career', 0.012740642002274236)
 
Cluster 3: 
('abuse', 0.10416090665419588)
('actor', 0.05842352025409666)
('american', 0.014093155674773808)
('article', 0.012288366593939193)
('ben', 0.012288366593939193)
('billion', 0.012288366593939193)
('called', 0.012288366593939193)
 
